<a href="https://colab.research.google.com/github/Chigaga/Dimimmings/blob/master/%22processing_dimmings%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install chart-studio

     |████████████████████████████████| 71kB 5.8MB/s 


In [3]:
import numpy as np
import pandas as pd
from pathlib import Path
import chart_studio.plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math

from IPython.display import HTML

py.sign_in("chigaga", "zPIazOZh5yTY1Uqn9z71")



In [4]:
tables_folder = Path('/content/drive/My Drive/Phd project/tables_folder')
sectors = pd.read_csv(tables_folder / '20111001.csv')
sectors

,Unnamed: 0,Unnamed: 1,area,brightness,interval,lat,lon,sector
0,166,0,5.831654e+06,-15.746248,0.666667,67.934821,-1.167568,3.0
1,166,1,5.827212e+06,-18.039982,0.666667,67.935499,-0.697511,3.0
2,166,2,5.822883e+06,-15.929602,0.666667,67.934658,-0.227295,3.0
3,166,3,5.826001e+06,-15.690197,0.666667,68.119173,-1.187040,3.0
4,166,4,5.821534e+06,-17.668350,0.666667,68.119908,-0.713227,3.0
...,...,...,...,...,...,...,...,...
4585231,599,13500,5.592571e+06,-49.504417,4.616667,87.924363,52.945970,5.0
4585232,599,13501,5.586769e+06,-66.715240,4.616667,87.779318,55.664713,5.0
4585233,599,13502,5.581051e+06,-57.630424,4.616667,87.629845,58.047199,5.0
4585234,599,13503,5.593432e+06,-0.000000,4.616667,88.022665,57.071914,5.0


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preparing data for plotting

Creating new columns in the frame

In [5]:
##we drop unuseful columns
sectors.rename({"Unnamed: 0":"a","Unnamed: 1":"b" }, axis="columns", inplace=True)
sectors.drop(["a","b"], axis=1, inplace=True)

#time ffom the first dimming appearance
sectors['time'] = sectors['interval']-sectors['interval'].min()
sectors['time'] = np.around(sectors['time'],decimals=3)
sectors['lon_class'] = np.around(sectors['lon'],decimals=-1)
sectors['number'] = 1 #to count pixels for mean brightness

#adding distance information
deg_dist = 12.17 #distance of 1 degree in Mm
sectors['ring'] = (90-sectors["lat"]).apply(np.ceil) 
sectors['ring'] = sectors['ring']-sectors['ring'].min()+1
sectors['dist'] = sectors['ring']*deg_dist

#splitting the frame for sectors back again - double number not to mix with distance plot
sec11 = sectors.query('sector == 1.0')
sec22 = sectors.query('sector == 2.0')
sec33 = sectors.query('sector == 3.0')
sec44 = sectors.query('sector == 4.0')
sec55 = sectors.query('sector == 5.0')

sectors

,area,brightness,interval,lat,lon,sector,time,lon_class,number,ring,dist
0,5.831654e+06,-15.746248,0.666667,67.934821,-1.167568,3.0,0.00,-0.0,1,23.0,279.91
1,5.827212e+06,-18.039982,0.666667,67.935499,-0.697511,3.0,0.00,-0.0,1,23.0,279.91
2,5.822883e+06,-15.929602,0.666667,67.934658,-0.227295,3.0,0.00,-0.0,1,23.0,279.91
3,5.826001e+06,-15.690197,0.666667,68.119173,-1.187040,3.0,0.00,-0.0,1,22.0,267.74
4,5.821534e+06,-17.668350,0.666667,68.119908,-0.713227,3.0,0.00,-0.0,1,22.0,267.74
...,...,...,...,...,...,...,...,...,...,...,...
4585231,5.592571e+06,-49.504417,4.616667,87.924363,52.945970,5.0,3.95,50.0,1,3.0,36.51
4585232,5.586769e+06,-66.715240,4.616667,87.779318,55.664713,5.0,3.95,60.0,1,3.0,36.51
4585233,5.581051e+06,-57.630424,4.616667,87.629845,58.047199,5.0,3.95,60.0,1,3.0,36.51
4585234,5.593432e+06,-0.000000,4.616667,88.022665,57.071914,5.0,3.95,60.0,1,2.0,24.34


preparing a frame grouped by sectors for plotting

In [12]:
#summing all values for finding brightness and area  (ee)
sum_by_sec = sectors.groupby(['time','sector']).sum().reset_index()
sum_by_sec['mean br'] = sum_by_sec['brightness']/sum_by_sec['number']
sum_by_lon = sectors.groupby(['time','lon_class']).sum().reset_index()

sum_by_sec_dist = sectors.groupby(['time','sector','ring','dist']).size().to_frame('count').reset_index()
sum_by_sec_lon

,time,sector,ring,dist,count
0,0.00,3.0,16.0,194.72,18
1,0.00,3.0,17.0,206.89,23
2,0.00,3.0,18.0,219.06,30
3,0.00,3.0,19.0,231.23,35
4,0.00,3.0,20.0,243.40,38
...,...,...,...,...,...
37237,3.95,5.0,7.0,85.19,71
37238,3.95,5.0,8.0,97.36,60
37239,3.95,5.0,9.0,109.53,30
37240,3.95,5.0,10.0,121.70,3


## Animation of area/brightness depending on the polar angle

In [ ]:
fig2 = px.area(sum_by_lon, x='lon_class', y='brightness', #color='time UT' ,
                  title='<b>Intensity decrease depending on longitude</b>', animation_frame="time",)#, line_close=True)
fig2.update_layout(
    xaxis_title="Longitude [deg]",
    yaxis_title='Brightness [DN]')
fig2.update_layout(xaxis = dict(range=[-180,180]),yaxis = dict(range=[0,-100000]) )
#fig3.update_xaxes(nticks=5)
fig2.show()

animation_br_plot_url = py.plot(fig2, filename='animation_br_time', auto_open=False,)
print(animation_br_plot_url)

# Leading edge analysis

Figure 0: distribution of pixels by distance

In [20]:
##if we want to check one time step
t = 1.09
time1 = sectors[sectors['time'] == t]
rr = time1
#sec_string = str(rr['sector'][0])
one_step = rr.groupby(['time','sector','dist']).size().to_frame('count').reset_index()
fig0 = px.line(one_step, x='dist', y='count', color='sector',#line_group="sector",range_x=[-180,180],
                title='<b>Distance of pixels for each sector after 1 hour from the dimming appearance </b>')#, line_close=True)
fig0.update_layout( xaxis_title="Distance from the center [Mm]",  yaxis_title='count')
fig0.update_xaxes(range=[0,500])
fig0.show()

one_time_plot_url = py.plot(fig0, filename='one_time', auto_open=False,)
print(one_time_plot_url)

https://plotly.com/~chigaga/70/


Figure 1: pixels by distance for all time steps

In [13]:
fig1 = px.line(sum_by_sec_dist, x='dist', y='count', color='sector',line_group="time",# range_x=[-180,180],
            title='<b>Distance of pixels for each sector </b>'  )#, line_close=True)
fig1.update_layout(xaxis_title="Distance from the center [Mm]",   yaxis_title='count')
fig1.update_xaxes(range=[0,400])
fig1.show()


Now we want to create a minimal number of pixels in each ring to be counted as leading edge: we take 10% of each ring's area. 

Figure 2: time profile for the leading edge

In [14]:
#area of the ring  - we know that radius of each ring is 6 pixels
percent = 0.1 #percent of area enought for the lead edge
sum_by_sec_dist['S'] = (2*sum_by_sec_dist['ring']*6-1)*math.pi*percent

sum_by_sec_dist['edge'] = np.where((sum_by_sec_dist['count'] >= sum_by_sec_dist['S']),sum_by_sec_dist['dist'], np.nan)
sum_by_sec_dist = sum_by_sec_dist.dropna()
distance=sum_by_sec_dist.groupby(['time','sector']).edge.agg([max]).reset_index()
#distance=sum_by_sec_dist.groupby(['time','sector'])['edge'].transform(max).reset_index()
distance

,time,sector,max
0,0.007,3.0,146.04
1,0.013,3.0,146.04
2,0.013,4.0,231.23
3,0.017,3.0,146.04
4,0.017,4.0,231.23
...,...,...,...
1884,3.950,1.0,292.08
1885,3.950,2.0,328.59
1886,3.950,3.0,279.91
1887,3.950,4.0,243.40


In [15]:
rr=distance
fig = px.line(rr, x='time', y='max', color='sector',# range_x=[-180,180],
                title='<b>Distance of pixels for each sector </b>')#, line_close=True)
fig.update_layout(xaxis_title="Distance from the center [Mm]", yaxis_title='count')
fig.update_xaxes(nticks=10,title_text="Time")
fig.show()

dist_plot_url = py.plot(fig, filename='dist', auto_open=False,)
print(dist_plot_url)

https://plotly.com/~chigaga/3/


In [16]:
sec1 = distance.query('sector == 1.0')
sec2 = distance.query('sector == 2.0')
sec3 = distance.query('sector == 3.0')
sec4 = distance.query('sector == 4.0')
sec5 = distance.query('sector == 5.0')

function to plot the time derivative

In [17]:
#mode: 'edge' or 'area' - to plot different parameters
def plotting_edge_rate(sec, mode, color,group,name):
  rr = sec
  x = rr['time']
  if mode=='edge':
    y = rr['max']
  else:
    y = rr['area']

  dy = np.zeros(y.shape,np.float)
  dy[0:len(y)-1] = np.diff(y)/np.diff(x)/3600
  dy[-1] = dy[-2]

  return (fig.add_trace(go.Scatter(x=x, y=dy,mode='lines',line = dict(color=color),legendgroup=group,name=name), row=2, col=1))

In [21]:
fig = make_subplots(rows=2, cols=1,shared_xaxes=True,)
fig.add_trace(go.Scatter(x=sec1['time'], y=sec1['max'],mode='lines', line = dict(color='#636EFA'),legendgroup="group1",name='edge 1'), row=1, col=1)
fig.add_trace(go.Scatter(x=sec2['time'], y=sec2['max'],mode='lines', line = dict(color='#EF553B'),legendgroup="group2",name='edge 2'), row=1, col=1)
fig.add_trace(go.Scatter(x=sec3['time'], y=sec3['max'],mode='lines', line = dict(color='#00CC96'),legendgroup="group3",name='edge 3'), row=1, col=1)
fig.add_trace(go.Scatter(x=sec4['time'], y=sec4['max'],mode='lines', line = dict(color='#AB63FA'),legendgroup="group4",name='edge 4'), row=1, col=1)
fig.add_trace(go.Scatter(x=sec5['time'], y=sec5['max'],mode='lines', line = dict(color='#FFA15A'),legendgroup="group5",name='edge 5'), row=1, col=1)

plotting_edge_rate(sec1, mode='edge', color='#636EFA',group = 'group1',name='rate 1')
plotting_edge_rate(sec2, mode='edge', color='#EF553B',group = 'group2',name='rate 2')
plotting_edge_rate(sec3, mode='edge', color='#00CC96',group = 'group3',name='rate 3')
plotting_edge_rate(sec4, mode='edge', color='#AB63FA',group = 'group4',name='rate 4')
plotting_edge_rate(sec5, mode='edge', color='#FFA15A',group = 'group5',name='rate 5')

fig.update_layout(title='<b>Leading edge and its rate for each sector in time</b>',width=800, height=700)
fig.update_yaxes(title_text="Leading edge [Mm]", row=1, col=1)
fig.update_yaxes(title_text="Rate [Mm/s]", row=2, col=1)
fig.update_xaxes(nticks=10,title_text="Time [h]")
fig.show()


lead_edge_plot_url = py.plot(fig, filename='lead_edge', auto_open=False,)
print(lead_edge_plot_url)

https://plotly.com/~chigaga/75/


## Time evolution of area and *brightness*